# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
def assert_cluster_checked():
    assert run_cluster.value is True, "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
assert_cluster_checked()
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
assert_cluster_checked()
client, dview, lview = hpc05.start_remote_and_connect(20, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
syst_pars = {'L_down' :  1000,
               'L_up' : 1000,
               'Lm' : 500,
               'Lx' : 1000,
               'a' :  15.625,
            'mu_from_bottom_of_spin_orbit_bands': True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.0)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,1.2]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = { "energy_gap":{"tol":1e-3},
                       "pfaffian":{}}

### Define data folder

In [ ]:
data_folder = 'topological_gap_vs_delta'

# Make ASS

In [ ]:
%autoreload
ASS = distributed_sns.AggregatesSimulationSet
ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

### Add additional dimensions

In [ ]:
def _params_Delta(syst_pars, params, Delta):
    params['Delta_right'] = Delta
    params['Delta_left'] = Delta
    return Delta

Deltas = np.linspace(0.05, 1.0, 11)

params_Delta = [ft.partial(_params_Delta,
                           Delta=_Delta
                           ) for _Delta in Deltas]

In [ ]:
ass.add_dimension("Delta", params_Delta)

### Make learner

In [ ]:
ass.make_balancing_learner(1000)
ass.load(data_folder, 1000)

In [ ]:
%%opts Image {+framewise}[aspect=1 colorbar=True]
kdims, plot_dict = ass.get_plot_dict(200, True, True)
hv.util.Dynamic(hv.HoloMap(plot_dict, kdims=kdims))

### Make runner with saver

In [ ]:
runner = adaptive.Runner(ass.get_balancing_learner(), executor=client)
ass.start_periodic_saver(runner, data_folder, interval=180)
runner.live_info()

# Plot

In [ ]:
[(l.npoints, l.loss()) for l in ass.learners]



In [ ]:
hv.extension('matplotlib')

In [ ]:
kdims, plot_dict = ass.get_plot_dict(200, True, True)
plot_dict = {k:v.redim.range(z= (0,0.0125)) for k,v in plot_dict.items()}

In [ ]:
%%opts Image {+framewise}[aspect=1 colorbar=True]
%%output filename=f"./{data_folder}/topological_gap_increasing_delta" fig='png'

hv.HoloMap(plot_dict, kdims=kdims)